# Import Packages

In [2]:
## Generic packages that are on your operating system
import glob
import os
import time

## Additional required packages
import numpy as np
import joblib
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt


# Required definitions

In [3]:
files = glob.glob('datasets/IMERG*_Nairobi.zarr')
ds_IMERG = xr.open_mfdataset(files,engine="zarr")

latitude_IMERG = ds_IMERG.latitude.values
longitude_IMERG = ds_IMERG.longitude.values

box_left = longitude_IMERG-0.05
box_right = longitude_IMERG+0.05
box_down = latitude_IMERG-0.05
box_up = latitude_IMERG+0.05


def bounding_box(region,points,min_x=box_left,max_x=box_right,min_y=box_down,max_y=box_up):

    """
    points: latitude, longitude
    x: longitude
    y: latitude
    """

    section_x = []
    section_y = []
    
    for x_min, x_max in zip(box_left,box_right):
        if region == 'J.K.I.A.':
            section_x.append(np.logical_and(points[1]<=x_max+0.01,points[1]>=x_min-0.01))
        else:
            section_x.append(np.logical_and(points[1]<=x_max,points[1]>=x_min))
            
    for y_min, y_max in zip(box_down,box_up):    
        section_y.append(np.logical_and(points[0]<=y_max,points[0]>=y_min))

    idx = np.squeeze(np.argwhere(np.logical_and(np.array(section_x),np.array(section_y))))

    return idx

In [66]:
def return_as_xarray(data, ds_ref, percentiles):
    
    dims = ['percentile','bin','time','region']
    coords = {
            'percentile': percentiles,
             'bin': ds_ref.bin.values,
             'time':ds_ref.time.values,
             'region':ds_ref.region.values
         }
    da = xr.DataArray(data=data,
                         dims=dims,
                         coords=coords,
                      )
    return da

def MultiClassEncoder(data, n_classes=4,ex_prob=False):

    data_encoded = [xr.where(data==cl,1,0).expand_dims(dim={"bin": [cl]}) for cl in range(n_classes)]
    return xr.concat(data_encoded, dim="bin")

def get_bins(data, bins=[5,20,50], OBS=False):

    data = data.to_dataarray()
    dims = ['time','region']
    coords = {
         'time':data.time.values,
         'region':data.coords['variable'].values,
     }
    binned_data = np.searchsorted(bins, data).T
    binned_data = xr.DataArray(data=binned_data,
                         dims=dims,
                         coords=coords,
                      )
    return binned_data

    
def get_warning(binned_data, allowed_risk=None):
    
    if allowed_risk is None:
        Warnings = MultiClassEncoder(binned_data)
        return Warnings
    
    else:

        ex_prob = binned_data
        Warnings = np.zeros([len(allowed_risk),len(ex_prob.bin.values),
                             len(ex_prob.time.values),1])
        
        i=0
        for risk in allowed_risk:
            Warnings[i,ex_prob>=risk] = 1
            i+=1

        return return_as_xarray(Warnings, ex_prob, allowed_risk)

def get_hits_misses_far(Warnings_obs, Warnings):
    
    ## Only interested where both are 1
    Hits = Warnings_obs*Warnings

    ## Only interested for difference = 1
    False_alarms = xr.where((Warnings-Warnings_obs)<=0,0,(Warnings-Warnings_obs))

    ## Only interested for difference = -1
    Misses = xr.where((Warnings_obs-Warnings)<=0,0,(Warnings_obs-Warnings))

    ## Only interested where both are 0
    True_negatives =  np.abs(Warnings-1)*np.abs(Warnings_obs-1)

   
    Hits = Hits.expand_dims({'metric':['Hits']},axis=0)
    Misses = Misses.expand_dims({'metric':['Misses']},axis=0)
    False_alarms = False_alarms.expand_dims({'metric':['False_alarms']},axis=0)
    True_negatives = True_negatives.expand_dims({'metric':['True_negatives']},axis=0)


    return xr.concat([Hits,False_alarms,True_negatives,Misses], 'metric')

        
            


# Load in data

In [63]:
files = glob.glob('datasets/cGAN_50_202*_Nairobi.zarr')
ds_cGAN_50 = []
for file in files:
    ds_temp = xr.open_dataset(file,engine="zarr")
    if type(ds_temp['time'].values[0]) == np.float32:
        ds_temp['time'] = (ds_temp['time'].astype('datetime64[ns]')+np.timedelta64(1,'h')).astype('datetime64[D]').astype('datetime64[ns]')
    ds_cGAN_50.append(ds_temp) 
ds_cGAN_50 = xr.concat(ds_cGAN_50, 'time').sortby('time')
ds_cGAN_50 = ds_cGAN_50.sel(time=ds_cGAN_50.time.dt.year.isin([2023,2024]))

files = glob.glob('datasets/cGAN_202*_Nairobi.zarr')
ds_cGAN = []
for file in files:
    ds_temp = xr.open_dataset(file,engine="zarr")
    if type(ds_temp['time'].values[0]) == np.float32:
        ds_temp['time'] = (ds_temp['time'].astype('datetime64[ns]')+np.timedelta64(1,'h')).astype('datetime64[D]').astype('datetime64[ns]')
    ds_cGAN.append(ds_temp) 
ds_cGAN = xr.concat(ds_cGAN, 'time').sortby('time')
ds_cGAN = ds_cGAN.sel(time=ds_cGAN.time.dt.year.isin([2023,2024]))

files = glob.glob('datasets/IFS_202*_Nairobi.zarr')
ds_IFS = []
for file in files:
    ds_IFS.append(xr.open_zarr(file).drop_duplicates('time',keep='last'))

ds_IFS = xr.concat(ds_IFS,'time').sortby('time')
ds_IFS = ds_IFS.sel(time=ds_IFS.time.dt.year.isin([2023,2024]))

meta = pd.read_excel('SEWAA-data.xlsx',sheet_name='Gridded',index_col=0,nrows=2)
data = pd.read_excel('SEWAA-data.xlsx',sheet_name='Gridded',index_col=0,skiprows=[1,2])

ds = data.to_xarray().rename({'ID':'time'})

# Not sure if this is the best way but it works
ds['time'] = np.array([t[:4]+'-'+t[4:6]+'-'+t[-2:] for t in ds['time'].values.astype(str)],dtype='datetime64[ns]')
ds = ds.sel(time=ds.time.dt.year.isin([2023,2024]))


In [65]:
ds

<xarray.Dataset> Size: 20kB
Dimensions:    (time: 365)
Coordinates:
  * time       (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
Data variables:
    KABETE     (time) float64 3kB 0.0 5.134 1.323 1.039 ... 1.221 0.0 0.857 0.0
    EASTLEIGH  (time) float64 3kB 0.0 5.14 2.158 1.158 ... 0.701 0.0 5.533 0.892
    J.K.I.A.   (time) float64 3kB 0.0 5.997 3.542 1.477 ... 0.0 0.0 5.381 2.31
    DAGORETTI  (time) float64 3kB 0.0 5.527 1.799 1.135 ... 0.0 0.039 0.912
    NGONG      (time) float64 3kB 0.0 6.381 1.788 1.125 ... 0.893 0.0 0.367 0.0
    WILSON     (time) float64 3kB 0.0 5.582 2.003 1.227 ... 0.0 0.0 0.0 0.507

In [46]:
val_data_x = {}

val_data_x['cGAN'] = ds_cGAN
val_data_x['cGAN_50'] = ds_cGAN
val_data_x['IFS'] = ds_IFS

val_valid_times = np.intersect1d(val_data_x['IFS'].time.values + np.timedelta64(1,'D'),
                                   ds.time.values)
val_data_x['cGAN'] = val_data_x['cGAN'].sel({'time':val_valid_times - np.timedelta64(1,'D')})
val_data_x['cGAN_50'] = val_data_x['cGAN_50'].sel({'time':val_valid_times - np.timedelta64(1,'D')})
val_data_x['IFS'] = val_data_x['IFS'].sel({'time':val_valid_times - np.timedelta64(1,'D')})

val_data_y = ds.sel({'time':val_valid_times})


In [47]:
val_data_y

<xarray.Dataset> Size: 5kB
Dimensions:    (time: 88)
Coordinates:
  * time       (time) datetime64[ns] 704B 2023-10-02 2023-10-03 ... 2023-12-30
Data variables:
    KABETE     (time) float64 704B 0.741 1.484 7.982 0.934 ... 1.221 0.0 0.857
    EASTLEIGH  (time) float64 704B 0.657 1.299 0.0 1.061 ... 0.0 0.701 0.0 5.533
    J.K.I.A.   (time) float64 704B 0.67 0.836 0.0 0.937 ... 0.0 0.0 0.0 5.381
    DAGORETTI  (time) float64 704B 0.74 1.313 3.155 0.728 ... 0.785 0.0 0.039
    NGONG      (time) float64 704B 0.92 1.359 0.582 0.651 ... 0.893 0.0 0.367
    WILSON     (time) float64 704B 0.652 1.317 1.385 0.801 ... 0.0 0.0 0.0 0.0

# Get logistic regression probabilities

In [67]:
model = 'cGAN'
region = 'KABETE'
hits_misses_far = {}
hits_misses_far[model] = {}

idx_region =  bounding_box(region,meta[region].values,min_x=box_left,max_x=box_right,min_y=box_down,max_y=box_up)
val_data_y_region = val_data_y[region].values
val_data_x_region = val_data_x[model].isel({'latlon':idx_region}).Nairobi.values

if val_data_x_region.shape[1]==1000:
    val_data_x_region = np.percentile(val_data_x_region,np.linspace(1,100,50),axis=1,method='weibull').T

logreg = joblib.load(f'models/{model}_{region}_1-day_fcst.pkl')

predicted_probabilities = xr.DataArray(data=logreg.predict_proba(val_data_x_region).T[:,:,None],
                                       dims = ['bin','time','region'],
                                       coords = {
                                           'time':val_data_y.time.values,
                                           'bin':np.arange(4),
                                           'region':[region]
                                       }
                                      )

ds_bin = get_bins(ds,bins=[5,20,40])
warnings_obs = get_warning(ds_bin)
warnings = get_warning(predicted_probabilities, [0.3])
hits_misses_far[model][region] = get_hits_misses_far(warnings_obs, warnings)
